# MILP Formulation from Nathan Kallus' Paper (Problem 4)

In [1]:
import gurobipy as gp
from gurobipy import GRB
import sys
import math
import numpy as np
import pandas as pd
import random

## Model specifications
#### Since we chose to modify the formulation to a certain extent, these variables simply allow us to revert back to the original model
- delta_include: If true, then constraint 4c from original formulation holds. If false, only the added constraint that gamma[p] need to add to 1 holds
- different_Cp = If true, then we have different sets of branching choices for every non-leaf node (like original formulation). If false, then we have a static set C for all nodes

In [2]:
# -- This class is an alternative to solving the right/left ancestor problem --
"""
INPUT: d = depth of tree (which includes root node), so d = 2 would make a tree with {1, 2, 3}
RELEVANT FUNCTIONS:
- get_right_left: For all leaf nodes, returns its right and left ancestors in a dictionary 
                  of {(p, q): 1 or -1 if q is right or left ancestor respectively}
"""
class Tree:
    def __init__(self, d):
        self.depth = d
        self.nodes = list(range(1, 2**(d-1)))
        self.leaves = list(range(2**(d-1), 2**d))
        self.ancestor_rl = {}
    
    def get_left_children(self, n):
        if n in self.nodes:
            return int(2*n)
        else:
            raise Exception ('Invalid node n')
    
    def get_right_children(self, n):
        if n in self.nodes:
            return int(2*n+1)
        else:
            raise Exception ('Invalid node n')
    
    def get_parent(self, n):
        if (n in self.nodes) | (n in self.leaves):
            return int(math.floor(n/2))
        else:
            raise Exception ('Invalide node n')
    
    def get_ancestors(self, direction, n):
        current = n
        ancestors = []
        while current != 1:
            current_buffer = self.get_parent(current)
            if direction == 'r':
                if self.get_right_children(current_buffer) == current:
                    ancestors.append(current_buffer)
            else:
                if self.get_left_children(current_buffer) == current:
                    ancestors.append(current_buffer)
            current = current_buffer
        return ancestors
    
    def get_right_left(self):
        for i in self.leaves:
            right = self.get_ancestors('r', i)
            for j in right:
                self.ancestor_rl[(i, j)] = 1
            left = self.get_ancestors('l', i)
            for j in left:
                self.ancestor_rl[(i, j)] = -1
        return self.ancestor_rl

# Evaluating Synthetic Data

In [3]:
def open_file(file_path):
    df = pd.read_csv(file_path)
    #train_X = df.iloc[:, :25]
    train_X = df[['AGE2', 'AGE3', 'RVISINF', 'RSBP2', 'RSBP3', 'RSBP4', 'RDEF3', 'RDEF4', 'RDEF5', 'RCONSC1', 'RCONSC2']]
    real = df[['y0', 'y1', 'y2', 'y3', 'y4', 'y5']]
    train_t = df['t']
    train_y = df['y']
    return train_X, train_t, train_y, real

In [5]:
def open_file_v2(file_path):
    df = pd.read_csv(file_path)
    train_X = df.iloc[:, :3]
    real = df.iloc[:, 3:5]
    train_t = df.iloc[:, 5]
    train_y = df.iloc[:, 7]
    return train_X, train_t, train_y, real

In [4]:
def datapoint_tree(node, i, test_X, test_real, test_t):
    if node in L: #if datapoint has reached leaf node, calculate error
        index = treatments[node]
        ideal_outcome = max(test_real.iloc[i, :])
        difference = ideal_outcome - test_real.iloc[i, index]
        if difference == 0:
            count_optimal = 1
        else:
            count_optimal = 0
        
        if index == test_t[i]:
            same_treatment = 1
        else:
            same_treatment = 0
        return difference, count_optimal, same_treatment
    if test_X.iloc[i, branching[node]] <= 0: # go left (node 2)
        return datapoint_tree(tree.get_left_children(node), i, test_X, test_real, test_t)
    else:
        return datapoint_tree(tree.get_right_children(node), i, test_X, test_real, test_t)

In [5]:
def get_metrics(test_X, test_real, test_t):
    difference = 0
    count_optimal = 0
    count_same = 0
    for i in range(len(test_X)):
        diff, optimal, treat = datapoint_tree(1, i, test_X, test_real, test_t)
        difference += diff
        count_optimal += optimal
        count_same += treat
    return difference, float(count_optimal)/len(test_X), float(count_same)/len(test_X)

## Declaring variables determined a-priori

### a) Specfiying Input to Model
- m treatments indexed by t {1,..., m}
- n datapoints indexed by i {(X1, T1, Y1), ..., (Xn, Tn, Yn)} 
- d: depth of decision tree
- n_min: minimum number of datapoints of each treatment in node p
- num_features
- num_cuts

In [6]:
def run_tree(depth, bert):
    delta_include = False
    different_Cp = False
    bertsimas = bert

    m = {0, 1, 2, 3, 4, 5}
    n = len(train_X)
    d = int(depth)
    n_min = 0
    num_features = 2
    num_cuts = 2

    """# ---- CONSTRUCTING COMPLETE BINARY TREE ----
    # - P = number of nodes in the tree
    # - L_c = set of non-leaf nodes
    # - L = set of leaf ndoes"""

    P = 2**d
    L_c = set(range(1, 2**(d-1)))
    L = set(range(2**(d-1), P))

    # - ancestors: dictionary {leaf nodes: {set of ancestors}}
    ancestors = {}
    for p in L:
        ancestors[p] = [math.floor(p/(2**j)) for j in range(1, d)]

        # Alternative way of retrieving right/left ancestors
    tree = Tree(d)
    right_left = tree.get_right_left()


    """
    - C[p]: finite set of cuts on node p--determined apriori {(l, theta)}
        Representation: dictionary {non-leaf node: list of (l, theta)}
        Require a list instead of set so it's ordered (indexed easily)


    From Kallus' paper, he wants us to:
    1. For each l in [d], sort data along x_l
    2a. For each non-leaf node, pick #features from [d] randomly
    2b. Set J = {1, (n-1/#cuts), ..., n-1} in decreasing order of cuts
    2c. Set Cp = {(l, midpoint between the two buckets in J) for all dimensions chosen and for all j in J}

    Make version of ALG3 to take all features"""


    if different_Cp:
        pass
    else:
        # -- BINARY COVARIATES --> Create a finite set of cuts for C for all features --
        C = []
        for i in range(len(train_X.columns)):
            C.append((i, 0))


    """ --- OTHER DATA --- 
    BIG M Constraints:
    - Ybar
    - Ymax
    - M

    BINARY ENCODING FOR CUTS
    - k_p: dictionary {non-leaf node p: k_p value}
    - Z_p: dictionary {non-leaf node p: k_p x |C_p| 2d matrix}
    """

    # ---- Big M Constraints ----
    # Ybar is merged into data, but it could not be. ybar is a numpy array
    minimum = min(train_y)

    ybar = train_y - minimum
    #data['ybar'] = ybar

    # Ymax
    ymax = max(ybar)

    # M
    # Find all sums for treatments 1, ..., m
    #treatment_counts = treatment.value_counts().to_list()
    unique, counts = np.unique(train_t, return_counts=True)
    #frequencies = numpy.asarray((unique, counts)).T
    M = np.array(counts)
    M -= len(L) * n_min
    M = max(M)
    
    model = gp.Model("Kallus")
    #model.params.TimeLimit = 3600

    # -- VARIABLE DECLARATION --

    # -- Variables to determine: gamma and lambda --
    # 1. gamma_p = choice of cut at node p ([0, 1]^C_p) (only applies to non-leaf node)
    #       - represent with a matrix gamma (|L_c| x |C_p|)

    gamma = model.addVars(L_c, len(C), vtype=GRB.BINARY, name='gamma') 
    # This assumes gamma is binary
    #gamma = model.addVars(L_c, len(C), vtype=GRB.BINARY, name='gamma') 

    # 2. lambda_pt = choice of treatment t at node p (only applies to leaf nodes L)
    #       - represent with a matrix lamb (|L| x m)
    lamb = model.addVars(L, m, vtype=GRB.BINARY, name='lamb')


    # -- Other Variables in Formulation --
    # 1. w_ip = membership of datapoint i in node p (only applies to leaf nodes L)
    #       - represent with a matrix w (n x |L|)

    w = model.addVars(n, L, lb=0, ub=1, name='w')
    # This assumes w is binary, when in reality it is continuous from 0-1
    #w = model.addVars(n, L, vtype=GRB.BINARY, name='w') # Original paper has this be a continuous variable

    # 2. mu_p = mean outcome of prescribed treatment in node p
    #       - represent with a matrix mu (|L|)
    mu = model.addVars(L, lb=0, name='mu') # define in constraint

    # 3. nu_ip = "effect" of treatment in node p by multiplying mu and w
    #       - represent with a matrix nu (n x |L|)
    nu = model.addVars(n, L, lb=0, name='nu')

    # 4. delta_p = forces only 1 choice of cut at node p
    #       - represent with a dictionary {non-leaf node p: 1d matrix of size k_p}
    #delta = model.addVars(L, k, vtype=GRB.BINARY, name='delta')

    # 5. Chi_i(gamma) = 1 if choice of cut induces datapoint i to go left on the cut gamma, 0 otherwise
    chi = model.addVars(L_c, n, vtype=GRB.BINARY, name='chi')

    if bertsimas:
        # 6. f_i
        f = model.addVars(n, lb=0, name='f')

        # 7. Beta_lt
        beta = model.addVars(L, m, lb=0, name='beta')

    theta = 0.5
    
        # --- OBJECTIVE FUNCTION ---
    if bertsimas:
        model.setObjective(theta * gp.quicksum(nu[i, p] for i in range(n) for p in L) 
                       - (1-theta) * gp.quicksum((train_y[i] - f[i]) * (train_y[i] - f[i]) for i in range(n)), GRB.MAXIMIZE)
    else:
        model.setObjective(gp.quicksum(nu[i, p] for i in range(n) for p in L), GRB.MAXIMIZE)


    # --- CONSTRAINTS ---
    # Constraint 4c (4b is done by definition of variables)
    if delta_include:
        for p in L_c:
            # need to do matrix multiplication somehow, but this might work?
            for j in range(k):
                model.addConstr(delta[p, j] == gp.quicksum(gamma[p, i] * z[j, i] for i in range(len(C))))

    # Additional constraint that gamma[p] adds up to 1
    # CHECKED
    for p in L_c:
        model.addConstr(gp.quicksum(gamma[p, i] for i in range(len(C))) == 1)

    # Add constraint Chi
    for i in range(n):
        for p in L_c:
            model.addConstr(chi[p, i] == gp.quicksum(gamma[p, j] for j in range(len(C)) if C[j][1] >= train_X.iloc[i, C[j][0]]))


    # Constraint 4d&e (Membership restriction from its ancestors) CHECKED
    for p in L:
        A_p = ancestors[p] #index ancestors of p
        for q in A_p:
            R_pq = right_left[(p, q)]
            for i in range(n):
                model.addConstr(w[i, p] <= (1+R_pq)/2 - R_pq * chi[q, i])


    #4e CHECKED
    for p in L:
        A_p = ancestors[p] #index ancestors of p
        for i in range(n):
            model.addConstr(w[i, p] >= 1 + gp.quicksum(-chi[q, i] for q in A_p if right_left[(p, q)] == 1)
                        + gp.quicksum(-1+chi[q, i] for q in A_p if right_left[(p, q)] == -1))


    # Constraint 4f
    # CHECKED
    for t in m:
        for p in L:
            model.addConstr(gp.quicksum(w[i, p] for i in range(n) if train_t[i] == t) >= n_min) #assuming the input comes in vector (Xi, Ti, Yi)
            # only add the datapoints that have been given treatment t

    # Constraints 4g&h (Linearization of nu)
    # CHECKED
    for p in L:
        for i in range(n):
            model.addConstr(nu[i, p] <= ymax * w[i, p])
            model.addConstr(nu[i, p] <= mu[p])
            model.addConstr(nu[i, p] >= mu[p] - ymax * (1-w[i, p]))

    # Constraint 4i (Choice of treatment applied to p)
    # CHECKED
    for p in L:
        model.addConstr(gp.quicksum(lamb[p, t] for t in m) == 1)

    # Constraint 4j&k (Consistency between lambda and mu)
    # CHECKED. There are some inconsistencies where some w don't appear, but this is because ybar is 0 (i.e. the minimum)
    for p in L:
        for t in m:
            model.addConstr(gp.quicksum(nu[i, p] - w[i, p] * ybar[i] for i in range(n) if train_t[i] == t) <= M*(1-lamb[p, t]))
            model.addConstr(gp.quicksum(nu[i, p] - w[i, p] * ybar[i] for i in range(n) if train_t[i] == t) >= M*(lamb[p, t]-1))
    #model.addConstr(lamb[2, 0] == 1)
    if bertsimas:
        for i in range(n):
            for p in L:
                for t in m:
                    if train_t[i] == t:
                        model.addConstr(f[i] - beta[p, t] <= M * (1-w[i, p]))
                        model.addConstr(f[i] - beta[p, t] >= M * (w[i, p]-1))
    
    #model.params.TimeLimit = 3600
    timeLimit = 3600
    oldSolutionLimit = model.Params.SolutionLimit
    model.Params.SolutionLimit = 1
    model.optimize()
    model.Params.TimeLimit = timeLimit - model.getAttr(GRB.Attr.Runtime)
    model.Params.SolutionLimit = oldSolutionLimit - model.Params.SolutionLimit
    model.optimize()
    
    
    #model.optimize()
    g = model.getAttr("X", gamma).items()
    l = model.getAttr("X", lamb).items()
    
    branching = {i[0][0]: i[0][1] for i in g if i[1] == 1.0}
    treatments = {i[0][0]: i[0][1] for i in l if i[1] == 1.0}
    
    return branching, treatments


In [7]:
"""kallus = pd.DataFrame(columns=['Dataset','Depth','P(Correct Treatment)','Test Error','Test % Optimal',
                               'Test % Same Treatment', 'Train Error', 'Train % Optimal', 
                               'Train % Same Treatment', 'Tree'])
bertsimas = pd.DataFrame(columns=['Dataset','Depth','P(Correct Treatment)','Test Error','Test % Optimal',
                               'Test % Same Treatment', 'Train Error', 'Train % Optimal', 
                                  'Train % Same Treatment', 'Tree'])"""


#bertsimas = pd.read_csv('bertsimas_athey_500.csv')

FileNotFoundError: [Errno 2] File b'bertsimas_athey_500.csv' does not exist: b'bertsimas_athey_500.csv'

In [ ]:
kallus = pd.DataFrame(columns=['Dataset','Depth','P(Correct Treatment)','Test Error','Test % Optimal',
                               'Test % Same Treatment', 'Train Error', 'Train % Optimal', 
                               'Train % Same Treatment', 'Tree'])


prob = 0.5
bert = False

depths = [1, 2, 3]
datasets = [1, 2, 3, 4, 5]
#probs = [0.1, 0.5, 0.9]

for dataset in datasets:
    for d in depths:
        train_filepath = 'data/IST_2000_binary/data_train_enc_' + str(dataset) + '.csv'
        test_filepath = 'data/IST_2000_binary/data_test_enc_' + str(dataset) + '.csv'
        train_X, train_t, train_y, train_real = open_file(train_filepath)
        branching, treatments = run_tree(d, bert)

        tree = Tree(d)
        L = set(range(2**(d-1), 2**d))
        test_X, test_t, test_y, test_real = open_file(test_filepath)
        error, pct, acc = get_metrics(test_X, test_real, test_t)
        error1, pct1, acc1 = get_metrics(train_X, train_real, train_t)

        tree_stats = 'branching = ' + str(branching) + ', treatments = ' + str(treatments)
        row = [dataset, d, prob, error, pct*100, acc*100, error1, pct1*100, acc1*100, tree_stats]
        print(row)
        if bert:
            bertsimas.loc[len(bertsimas)] = row
        else:
            kallus.loc[len(kallus)] = row


Using license file /Users/nathanjo/gurobi.lic
Academic license - for non-commercial use only - expires 2021-02-17
Changed value of parameter SolutionLimit to 1
   Prev: 2000000000  Min: 1  Max: 2000000000  Default: 2000000000
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8019 rows, 4007 columns and 24402 nonzeros
Model fingerprint: 0x3fba9f65
Variable types: 4001 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+02]
Presolve removed 8019 rows and 4007 columns
Presolve time: 0.05s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.07 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 1307.02 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.307017543860e+03, best bound 1.30701

In [ ]:
kallus.to_csv('Results_IST_binary/kallus_2000.csv')

In [11]:
dataset = 4
d = 2
bert = False
train_filepath = 'data/IST_2000/data_train_enc_' + str(dataset) + '.csv'
test_filepath = 'data/IST_2000/data_test_enc_' + str(dataset) + '.csv'
train_X, train_t, train_y, train_real = open_file(train_filepath)
branching, treatments = run_tree(d, bert)

tree = Tree(d)
L = set(range(2**(d-1), 2**d))
test_X, test_t, test_y, test_real = open_file(test_filepath)
error, pct, acc = get_metrics(test_X, test_real, test_t)
error1, pct1, acc1 = get_metrics(train_X, train_real, train_t)

tree_stats = 'branching = ' + str(branching) + ', treatments = ' + str(treatments)
row = [dataset, d, error, pct*100, acc*100, error1, pct1*100, acc1*100, tree_stats]
print(row)

Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 22039 rows, 10039 columns and 99696 nonzeros
Model fingerprint: 0xb0b31bf7
Variable types: 8002 continuous, 2037 integer (2037 binary)
Coefficient statistics:
  Matrix range     [3e-02, 6e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e-01, 6e+02]
Presolve removed 8034 rows and 4022 columns
Presolve time: 0.58s
Presolved: 14005 rows, 6017 columns, 79167 nonzeros
Variable types: 4002 continuous, 2015 integer (2015 binary)

Root relaxation: objective 1.897781e+03, 7002 iterations, 0.98 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1897.78083    0  568          - 1897.78083  

In [76]:
tree = Tree(d)
L = set(range(2**(d-1), 2**d))
test_X, test_t, test_y, test_real = open_file(test_filepath)
error, pct, acc = get_metrics(test_X, test_real, test_t)
error1, pct1, acc1 = get_metrics(train_X, train_real, train_t)
print(error, pct, acc)
print(error1, pct1, acc1)

267.9863312540136 0.4446 0.5066
21.252969854711775 0.354 0.5


In [77]:
tree_stats = 'branching = ' + str(branching) + ', treatments = ' + str(treatments)
row = [dataset, d, prob, error, pct*100, acc*100, error1, pct1*100, acc1*100, tree_stats]
if bert:
    bertsimas.loc[len(bertsimas)] = row
else:
    kallus.loc[len(kallus)] = row

[5, 3, 0.9, 267.9863312540136, 44.46, 50.660000000000004, 21.252969854711775, 35.4, 50.0, 'branching = {1: 10, 2: 16, 3: 0}, treatments = {4: 1, 5: 1, 6: 1, 7: 1}']


In [78]:
print(bertsimas)
bertsimas.to_csv('bertsimas_athey_500.csv', index=False)

    Dataset  Depth  P(Correct Treatment)  Test Error  Test % Optimal  \
0         1      2                   0.5   53.227602           76.83   
1         1      3                   0.5  123.833930           66.00   
2         1      3                   0.5  123.833930           66.00   
3         2      2                   0.5   24.915258           84.31   
4         2      3                   0.5   24.915258           84.31   
5         3      2                   0.5  166.280038           58.61   
6         3      3                   0.5   57.544135           77.50   
7         4      2                   0.5  432.810829           24.05   
8         4      3                   0.5  231.607156           44.69   
9         5      2                   0.5  189.645268           55.55   
10        5      3                   0.5   36.072978           82.59   
11        1      2                   0.9  224.473595           49.78   
12        1      3                   0.9  224.473595           4

### Summary of Performance for Athey's Synthetic Data

In [247]:
# SUMMARY: KALLUS on Athey
summary = {'Method': ['Dataset 1', 'Dataset 1', 'Dataset 2', 'Dataset 2', 'Dataset 3',
                     'Dataset 3', 'Dataset 4', 'Dataset 4', 'Dataset 5', 'Dataset 5', 'Dataset 1', 
                      'Dataset 1', 'Dataset 2', 'Dataset 2', 'Dataset 3',
                     'Dataset 3', 'Dataset 4', 'Dataset 4', 'Dataset 5', 'Dataset 5'],
           'Depth': [2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3],
           'P(Correct Treatment)': ['0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.9', '0.9',
                                   '0.9', '0.9', '0.9', '0.9', '0.9', '0.9', '0.9', '0.9'],
          'Error': [74.67, 74.67, 46.40, 79.86, 145, 109.21, 18.15, 18.15, 152.55, 90.47, 199.70, 238.67, 219.02, 
                    239.10, 201.21, 201.21, 99.23, 88.12, 196.07, 156.84],
          '% Classified': [71.97, 71.97, 78.49, 71.11, 61.22, 68.03, 88.33, 88.33, 60.41, 70.03, 53.35,
                          47.49, 49.91, 47.49, 53.42, 53.42, 71.19, 74.11, 54.12, 58.02]}
summary = pd.DataFrame(summary)
summary.to_csv('kallus_tree_athey.csv', index=False)

In [248]:
# SUMMARY: Bertsimas on Athey
summary = {'Method': ['Dataset 1', 'Dataset 1', 'Dataset 2', 'Dataset 2', 'Dataset 3',
                     'Dataset 3', 'Dataset 4', 'Dataset 4', 'Dataset 5', 'Dataset 5', 'Dataset 1', 
                      'Dataset 1', 'Dataset 2', 'Dataset 2', 'Dataset 3',
                     'Dataset 3', 'Dataset 4', 'Dataset 4', 'Dataset 5', 'Dataset 5'],
           'Depth': [2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3],
           'P(Correct Treatment)': ['0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.9', '0.9',
                                   '0.9', '0.9', '0.9', '0.9', '0.9', '0.9', '0.9', '0.9'],
          'Error': [74.67, 74.67, 46.40, 55.17, 145, 109.21, 18.15, 18.15, 152.55, 90.47, 199.70, 154.07, 219.02, 
                    180.83, 201.21, 116.75, 99.23, 88.12, 196.07, 156.84],
          '% Classified': [71.97, 71.97, 78.49, 76.49, 61.22, 68.03, 88.33, 88.33, 60.41, 70.03, 53.35,
                          62.17, 49.91, 53.91, 53.42, 71.59, 71.19, 74.11, 54.12, 58.02]}
summary = pd.DataFrame(summary)
summary.to_csv('bertsimas_tree_athey.csv', index=False)


In [37]:
def datapoint_tree_avg(node, i):
    if node in L: #if datapoint has reached leaf node, calculate error
        return node
    if train_X.iloc[i, branching[node]] <= 0: # go left (node 2)
        return datapoint_tree_avg(tree.get_left_children(node), i)
    else:
        return datapoint_tree_avg(tree.get_right_children(node), i)

In [ ]:
summation = {20: 0, 21: 0, 30: 0, 31: 0}
count = {20: 0, 21: 0, 30: 0, 31: 0}

for i in range(n):
    leaf_node = datapoint_tree_avg(1, i)
    index = str(leaf_node) + str(train_t[i])
    summation[int(index)] += train_y[i]
    count[int(index)] += 1
    
avg = {}
for i in summation:
    avg[i] = float(summation[i]) / count[i]

print(avg)

In [49]:
summation = {2: 0, 3: 0}
count = {2: 0, 3: 0}

for i in range(n):
    leaf_node = datapoint_tree_avg(1, i)
    if train_t[i] == treatments[leaf_node]:
        summation[leaf_node] += train_y[i]
        count[leaf_node] += 1
    
avg = {}
for i in summation:
    avg[i] = float(summation[i]) / count[i]

print(avg)

{2: 0.39851651102745433, 3: 0.34277517675515534}


In [59]:
N = L.union(L_c)
print(N)

model = gp.Model("Nathan")

# -- VARIABLE DECLARATION --

# -- Variables to determine: gamma and lambda --
# 1. gamma_p = choice of cut at node p ([0, 1]^C_p) (only applies to non-leaf node)
#       - represent with a matrix gamma (|L_c| x |C_p|)

gamma = model.addVars(L_c, len(C), vtype=GRB.BINARY, name='gamma') 
# This assumes gamma is binary
#gamma = model.addVars(L_c, len(C), vtype=GRB.BINARY, name='gamma') 

# 2. lambda_pt = choice of treatment t at node p (only applies to leaf nodes L)
#       - represent with a matrix lamb (|L| x m)
lamb = model.addVars(L, m, vtype=GRB.BINARY, name='lamb')


# -- Other Variables in Formulation --
# 1. w_ip = membership of datapoint i in node p (only applies to leaf nodes L)
#       - represent with a matrix w (n x |L|)

c = model.addVars(n, N, vtype=GRB.BINARY, name='c')
# This assumes w is binary, when in reality it is continuous from 0-1
#w = model.addVars(n, L, vtype=GRB.BINARY, name='w') # Original paper has this be a continuous variable

# 2. mu_p = mean outcome of prescribed treatment in node p
#       - represent with a matrix mu (|L|)
v = model.addVars(n, L_c, vtype=GRB.BINARY, name='v') # define in constraint

# 3. nu_ip = "effect" of treatment in node p by multiplying mu and w
#       - represent with a matrix nu (n x |L|)
w = model.addVars(n, vtype=GRB.BINARY, name='w')

{1, 2, 3}


In [ ]:


"""if different_Cp:
# ---- K and Z if we followed the definition of C_p from the paper -----
    for p in L_c:
        k[p] = math.ceil(math.log2(len(C[p])))

    z = {}
    for p in L_c:
        matrix = np.zeros((k[p], len(C[p])))
        print(matrix.shape)
        for i in range(1, k[p]+1):
            for j in range(1, len(C[p])+1):
                if math.floor(j/(2**i)) % 2 == 1: # odd number
                    z[i-1, j-1] = 1
                else:
                    z[i-1, j-1] = 0
        z[p] = matrix

else:
    # ---- K and Z if we had constant C for all nodes ----
    k = math.ceil(math.log2(len(C)))
    z = np.zeros((k, len(C)))
    for i in range(1, k+1):
        for j in range(1, len(C)+1):
            if math.floor(j/(2**i)) % 2 == 1: # odd number
                z[i-1, j-1] = 1
            else:
                z[i-1, j-1] = 0
print(z)"""